In [1]:
# %pip install langchain
# %pip install langchain-community
# %pip install pypdf
# %pip install sentence-transformers
# %pip install langchain
# %pip install langchain-community
# %pip install pinecone
# ! pip install openai


In [23]:
# %pip install sentence-transformers

In [1]:
import langchain
from langchain.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pypdf
from langchain_community.embeddings import HuggingFaceEmbeddings

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os

In [3]:
# !pip install pinecone
import pinecone
print(pinecone.__file__)


C:\Users\LENOVO\miniconda3\Lib\site-packages\pinecone\__init__.py


In [4]:
from pinecone import Pinecone, ServerlessSpec
#  api key of pincone vector database 
pc = Pinecone(api_key="pcsk_6F5Nao_NZDq1fx4jy9JJcqVitpWrvJzeDm7RJwpFHGKPDp3gzhYGNqPxzAXTbY5qnD5bMW")
#  creating index for storing data in vector form
index = pc.Index("medicalbot")

# print(pc.list_indexes())

# # You can now upsert/query
# index.upsert(vectors=[("id1", [0.1, 0.2, 0.3])])




In [5]:
#  create a function to fetch data from pdf
import fitz 
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_text = extract_text_from_pdf("C:/AI-project/AI/AI/medical_chatbot/data/train-15-120.pdf")

In [6]:
#  using hugging face model for embeddings data
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast & good quality
chunks = [pdf_text[i:i+500] for i in range(0, len(pdf_text), 500)]  # break into chunks

embeddings = model.encode(chunks).tolist()


In [7]:
#  save embedding data into vector db
from pinecone import Pinecone
pc = Pinecone(api_key="pcsk_6F5Nao_NZDq1fx4jy9JJcqVitpWrvJzeDm7RJwpFHGKPDp3gzhYGNqPxzAXTbY5qnD5bMW")
index = pc.Index("medicalbot")

vectors = [(f"id{i}", embeddings[i], {"text": chunks[i]}) for i in range(len(chunks))]
index.upsert(vectors=vectors)


{'upserted_count': 922}

In [28]:
#  asking question based on data
query = "what is treatment  gallblader cancer?"
query_embedding = model.encode([query])[0].tolist()

results = index.query(vector=query_embedding, top_k=3, include_metadata=True)

# retrieved_texts = [match['metadata']['text'] for match in results['matches']]


In [29]:
#  matching data of query
for match in results['matches']:
    print(match['score'], match['metadata']['text'])


0.647527337 r.”
In Current Surgical Therapy, edited by John L. Cameron,
sixth ed. St Louis: Mosby, 1998, pp.462-468.
“Gallbladder Carcinoma.” In Clinical Oncology, edited by
Abeloff, Martin D., second ed. New York: Churchill Liv-
ingstone, 2000, pp.1730-1737.
OTHER
National Cancer Institute Cancer Trials web site. <http://cancer-
trials.nci.nih.gov/system>. <http://www.cancertrials.com>.
Kevin O. Hwang, M.D.
Gallbladder disease see Cholecystitis
GALE ENCYCLOPEDIA OF MEDICINE 2
1380
Gallbladder cancer
Gallbl
0.642518342 “Gallbladder Carcinoma.” In Clinical Oncology, edited by
Abeloff, Martin D., second ed. New York: Churchill Liv-
ingstone, 2000, pp.1730-1737.
OTHER
National Cancer Institute Cancer Trials web site. <http://cancer-
trials.nci.nih.gov/system>. <http://www.cancertrials.com>.
Kevin O. Hwang, M.D.
Gallbladder disease see Cholecystitis
GALE ENCYCLOPEDIA OF MEDICINE 21380
Gallbladder cancer
0.642518342 “Gallbladder Carcinoma.” In Clinical Oncology, edited by
Abeloff, Martin D.

In [25]:
 #  getting answers
from transformers import pipeline

qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")

context = " ".join(retrieved_texts)
result = qa_model(question=query, context=context)

print("Answer:", result['answer'])
# print(chunks)


Device set to use cpu


Answer: Galactosemia is an inherited disease
